In [56]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *
from collections import Counter

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/.pyenv/versions/venv/lib/python3.8/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['record']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
experiment = Experiment.objects.get(name='infocompare')
runs = experiment.equilibriumsolverrun_set.all()

In [68]:
def analyze_checkpoint(checkpoint):
    record = dict()
    samples = checkpoint.evaluation.samples
    welfares = np.zeros(len(samples['rewards']['0']))
    revenues = np.zeros(len(samples['rewards']['0']))
    for player in range(checkpoint.equilibrium_solver_run.game.num_players):
        player = str(player)
        rewards = pd.Series(samples['rewards'][player])
        payments = pd.Series(samples['payments'][player])
        record[f'p{player}_utility'] = rewards.mean()
        record[f'p{player}_payment'] = payments.mean()
        welfares += rewards + payments
        revenues += payments
    record['total_welfare'] = welfares.mean()
    record['total_revenue'] = revenues.mean()

    arr = np.array(samples['allocations']['0']).astype(int) # TODO: only player 0's allocation.
    c = Counter(tuple(map(tuple, arr)))
    record['common_allocations'] = c.most_common(5)
    
    return record
    # print(welfares.mean(), revenues.mean())

def find_best_checkpoint(run):
    ev_df = parse_run(run, 9_000_000)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    approx_nash_conv = ev_df.groupby('t')['ApproxNashConv'].first().min()
    best_checkpoint = get_checkpoint(experiment.name, run.name, t=best_t)
    return best_checkpoint, approx_nash_conv

In [80]:
records = []
for game in ['large_game_2']:
    print(game)
    for run in tqdm(runs.filter(game__name=f'{game}.json')):
        print(run.config['seed'])
        checkpoint, approx_nash_conv = find_best_checkpoint(run)
        record = analyze_checkpoint(checkpoint)
        record['approx_nash_conv'] = approx_nash_conv
        records.append(record)
    print()
df = pd.DataFrame.from_records(records)

large_game_2


/tmp/ipykernel_28139/445496683.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for run in tqdm(runs.filter(game__name=f'{game}.json')):


  0%|          | 0/9 [00:00<?, ?it/s]

101
105
106
100
102
109
108
103
107



In [70]:
arr = checkpoint.evaluation.samples['rewards']['0']
data = (arr,)  
from scipy.stats import bootstrap
res = bootstrap(data, np.mean, confidence_level=0.9, n_resamples=500, method='percentile')
res

BootstrapResult(confidence_interval=ConfidenceInterval(low=131.40627455988943, high=131.77069214603736), standard_error=0.11127212808392974)

In [79]:
from collections import Counter
arr = np.array(checkpoint.evaluation.samples['allocations']['0']).astype(int)
c = Counter(tuple(map(tuple, arr)))
c.most_common(len(c))
# c.most_common(5)

[((2, 3, 0), 52473),
 ((3, 1, 1), 11801),
 ((0, 3, 2), 8351),
 ((0, 3, 3), 7247),
 ((3, 1, 0), 2769),
 ((1, 3, 0), 1600),
 ((3, 0, 1), 1533),
 ((1, 3, 2), 1481),
 ((3, 0, 2), 1375),
 ((1, 3, 1), 1299),
 ((2, 2, 1), 1172),
 ((2, 2, 2), 1074),
 ((2, 1, 1), 987),
 ((2, 0, 2), 924),
 ((2, 1, 2), 793),
 ((2, 0, 3), 743),
 ((0, 3, 1), 496),
 ((1, 2, 2), 490),
 ((2, 3, 1), 475),
 ((3, 0, 0), 297),
 ((2, 0, 1), 269),
 ((2, 2, 0), 233),
 ((0, 3, 0), 232),
 ((1, 2, 1), 210),
 ((1, 1, 3), 188),
 ((1, 2, 3), 184),
 ((2, 1, 0), 136),
 ((3, 1, 2), 127),
 ((0, 2, 2), 122),
 ((1, 1, 2), 113),
 ((2, 3, 2), 89),
 ((2, 1, 3), 79),
 ((2, 0, 0), 75),
 ((3, 2, 0), 70),
 ((1, 1, 1), 67),
 ((0, 2, 3), 66),
 ((3, 2, 1), 58),
 ((1, 0, 2), 42),
 ((1, 2, 0), 31),
 ((3, 3, 0), 31),
 ((1, 0, 3), 26),
 ((2, 2, 3), 25),
 ((3, 0, 3), 23),
 ((0, 2, 0), 17),
 ((3, 2, 2), 15),
 ((0, 1, 3), 15),
 ((3, 3, 1), 13),
 ((0, 2, 1), 12),
 ((0, 1, 2), 12),
 ((1, 0, 1), 10),
 ((0, 1, 1), 8),
 ((1, 1, 0), 8),
 ((1, 3, 3), 4),
 ((0,

In [73]:
# data

In [87]:
x = df.copy()
del x['common_allocations']
x

,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,approx_nash_conv
0,119.299547,260.433953,151.828569,223.894231,755.456300,484.328184,6.136778
1,110.283074,283.580826,142.192599,218.300201,754.356700,501.881027,4.596013
2,108.893819,280.877781,147.757828,219.723572,757.253000,500.601353,0.565621
3,88.345236,204.572365,151.576142,291.493358,735.987100,496.065723,10.670377
4,128.823059,303.109242,136.434884,189.175516,757.542700,492.284757,1.434662
5,118.516735,291.097365,144.563827,205.458673,759.636599,496.556037,1.962268
6,100.129506,278.194794,147.068753,229.939047,755.332100,508.133841,9.096962
7,112.897518,301.943282,131.527484,205.951316,752.319600,507.894599,3.092972
8,131.585773,293.050027,131.015494,190.298906,745.950200,483.348932,3.164699


In [86]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(df['common_allocations'])

0     [((3, 1, 0), 48502), ((3, 0, 1), 15740), ((0, 2, 3), 9116), ((2, 1, 1), 5702), ((2, 2, 0), 2920)]
1     [((3, 1, 0), 51059), ((3, 1, 1), 18585), ((0, 2, 3), 2915), ((1, 0, 3), 2265), ((1, 2, 0), 2030)]
2     [((3, 1, 0), 52859), ((3, 0, 2), 10133), ((3, 1, 1), 7589), ((0, 3, 3), 6406), ((0, 3, 0), 2300)]
3     [((2, 1, 0), 53382), ((0, 2, 2), 19880), ((0, 3, 1), 5349), ((2, 0, 1), 2844), ((0, 3, 2), 2425)]
4      [((3, 2, 0), 67191), ((0, 1, 3), 9668), ((0, 3, 3), 3597), ((0, 3, 2), 2545), ((3, 1, 0), 1947)]
5    [((3, 1, 0), 52412), ((0, 3, 3), 16876), ((3, 0, 2), 16313), ((3, 1, 1), 5705), ((3, 2, 0), 1191)]
6      [((3, 1, 0), 57387), ((3, 1, 1), 5073), ((3, 0, 1), 4793), ((2, 1, 1), 4097), ((0, 1, 3), 3189)]
7      [((3, 1, 0), 52099), ((0, 3, 3), 8713), ((3, 1, 1), 4749), ((3, 2, 0), 3904), ((1, 3, 3), 2717)]
8     [((2, 3, 0), 52473), ((3, 1, 1), 11801), ((0, 3, 2), 8351), ((0, 3, 3), 7247), ((3, 1, 0), 2769)]
Name: common_allocations, dtype: object

In [75]:
df['common_allocations']

0    [((3, 1, 0), 48502), ((3, 0, 1), 15740), ((0, ...
1    [((3, 1, 0), 51059), ((3, 1, 1), 18585), ((0, ...
2    [((3, 1, 0), 52859), ((3, 0, 2), 10133), ((3, ...
3    [((2, 1, 0), 53382), ((0, 2, 2), 19880), ((0, ...
4    [((3, 2, 0), 67191), ((0, 1, 3), 9668), ((0, 3...
5    [((3, 1, 0), 52412), ((0, 3, 3), 16876), ((3, ...
6    [((3, 1, 0), 57387), ((3, 1, 1), 5073), ((3, 0...
7    [((3, 1, 0), 52099), ((0, 3, 3), 8713), ((3, 1...
8    [((2, 3, 0), 52473), ((3, 1, 1), 11801), ((0, ...
Name: common_allocations, dtype: object

In [82]:
df['common_allocations'].loc[0]

[((3, 1, 0), 48502),
 ((3, 0, 1), 15740),
 ((0, 2, 3), 9116),
 ((2, 1, 1), 5702),
 ((2, 2, 0), 2920)]

In [83]:
df['common_allocations'].loc[1]

[((3, 1, 0), 51059),
 ((3, 1, 1), 18585),
 ((0, 2, 3), 2915),
 ((1, 0, 3), 2265),
 ((1, 2, 0), 2030)]

In [78]:
df['common_allocations'].loc[6]

[((3, 1, 0), 57387),
 ((3, 1, 1), 5073),
 ((3, 0, 1), 4793),
 ((2, 1, 1), 4097),
 ((0, 1, 3), 3189)]

In [114]:
from open_spiel.python.examples.ubc_utils import *

g_db = Game.objects.get(name='tin
g = load_game(g_db)
it = g.new_initial_state().child(0).child(0).child(2).child(1).information_state_tensor()

get_player_type_index(g_db.num_players, g_db.num_actions, g_db.num_products)
get_player_type(g_db.num_players, g_db.num_actions, g_db.num_products, it)

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/tiny.json
Done config parsing


MemoryError: std::bad_alloc

In [100]:
it[145:155]

[0.0, 500.0, 120.0, 80.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [108]:
it[146]

500.0

In [115]:
Game.objects.all()

<QuerySet [<Game: tiny.json>, <Game: large_game_2.json>, <Game: large_game_2_hide.json>]>